In [1]:
import yahoo_fin as yf
from yahoo_fin.stock_info import get_data
import time
import datetime
import pandas as pd
from yahoo_fin import stock_info as si

import tensorflow as tf 
import numpy as np
from tensorflow.keras import layers

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# getting the right tickers

some of this is shamelessly copying this:
https://levelup.gitconnected.com/how-to-get-all-stock-symbols-a73925c16a1b

things we did to clean data:

1. we removed the tickers with $.


2. we added exception handeling that tries similer strings to the input. (note that this is not implemented on the set itself but when fetching, leading to multiple requsts and potentialy to errors)


3. we have painstakingly went ahead and manualy found all exceptions and removed them if no reformating was possible.

In [2]:
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

symbols = set.union( sym1, sym2, sym3, sym4 )
del(sym1, sym2, sym3, sym4,df1, df2, df3, df4)

symbols

{'',
 'EL',
 'ALLE',
 'MXF',
 'REMX',
 'MLN',
 'PIE',
 'PINE',
 'SONX',
 'AMRN',
 'SLYV',
 'VHAQ',
 'BSJN',
 'LBAI',
 'MIMO.W',
 'STE',
 'ONB',
 'DNUT',
 'ICLK',
 'NGC',
 'CWEB',
 'SAI',
 'BJDX',
 'NSC',
 'MINV',
 'CZNC',
 'MTMT',
 'FSBC',
 'EZA',
 'LEGR',
 'NMRK',
 'NECB',
 'SBET',
 'ZIG',
 'FLRT',
 'YCS',
 'FSMB',
 'CFG$D',
 'PID',
 'RIGZ',
 'ERO',
 'SSU.W',
 'CBD',
 'GRMN',
 'IQSU',
 'EPP',
 'HYGW',
 'SBBA',
 'IP',
 'VIVE',
 'SPYG',
 'SAL',
 'BFI',
 'ZHDG',
 'FOXWW',
 'FEMA',
 'POET',
 'IVR',
 'IAE',
 'PSTL',
 'USI',
 'IFRA',
 'EDUT',
 'FRWAU',
 'HPKEW',
 'ALVOW',
 'AXTI',
 'DFS',
 'FLFR',
 'GEF',
 'GLRY',
 'MFIN',
 'SBUX',
 'JRO',
 'MDWD',
 'HNNA',
 'MNTS',
 'PWUPW',
 'WRAC.U',
 'GBR',
 'NFNT',
 'SPE',
 'AEL$B',
 'WCN',
 'NCMI',
 'GSLC',
 'API',
 'HCKT',
 'KREF',
 'CFFI',
 'VIOO',
 'CLVR',
 'FFWM',
 'SMFL',
 'PDN',
 'JOFFW',
 'EDNC',
 'BIGZ',
 'MAPS',
 'VUZI',
 'SPXT',
 'IVCAU',
 'SINT',
 'LOCL',
 'VISL',
 'AFTR.U',
 'AQB',
 'QLTA',
 'GTX',
 'HIL',
 'CENX',
 'MFGP',
 'TSLL',
 'BGR'

In [3]:
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if (len( symbol ) > 4 and symbol[-1] in my_list or "$" in symbol):
        del_set.add( symbol )
    else:
        sav_set.add( symbol )
        
p=pd.read_csv("shitlist")
p=list(p["0"])
del_set=del_set.union(p)
sav_set.difference_update(p)
print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1592 unqualified stock symbols...
There are 10606 qualified stock symbols...


In [4]:
#this just drops the odd empty string on the start
ticker_list=list(sav_set)[1:]
ticker_list

['EL',
 'ALLE',
 'MXF',
 'REMX',
 'MLN',
 'PIE',
 'PINE',
 'SONX',
 'AMRN',
 'VHAQ',
 'SLYV',
 'BSJN',
 'LBAI',
 'STE',
 'ONB',
 'DNUT',
 'ICLK',
 'NGC',
 'CWEB',
 'SAI',
 'BJDX',
 'NSC',
 'MINV',
 'CZNC',
 'MTMT',
 'FSBC',
 'EZA',
 'LEGR',
 'NMRK',
 'NECB',
 'SBET',
 'ZIG',
 'FLRT',
 'YCS',
 'FSMB',
 'PID',
 'RIGZ',
 'ERO',
 'CBD',
 'GRMN',
 'IQSU',
 'EPP',
 'HYGW',
 'SBBA',
 'VIVE',
 'IP',
 'SPYG',
 'SAL',
 'BFI',
 'ZHDG',
 'FEMA',
 'POET',
 'IVR',
 'IAE',
 'PSTL',
 'USI',
 'IFRA',
 'EDUT',
 'FRWAU',
 'AXTI',
 'DFS',
 'FLFR',
 'GEF',
 'GLRY',
 'MFIN',
 'SBUX',
 'JRO',
 'MDWD',
 'HNNA',
 'MNTS',
 'WRAC.U',
 'GBR',
 'NFNT',
 'SPE',
 'WCN',
 'NCMI',
 'GSLC',
 'API',
 'HCKT',
 'KREF',
 'CFFI',
 'VIOO',
 'CLVR',
 'FFWM',
 'SMFL',
 'PDN',
 'EDNC',
 'BIGZ',
 'MAPS',
 'VUZI',
 'SPXT',
 'IVCAU',
 'SINT',
 'LOCL',
 'VISL',
 'AFTR.U',
 'AQB',
 'QLTA',
 'GTX',
 'HIL',
 'CENX',
 'MFGP',
 'TSLL',
 'BGR',
 'GXTG',
 'MSGR',
 'SMIG',
 'DBEM',
 'IVRA',
 'AAPB',
 'VXX',
 'TTAC',
 'DHI',
 'RMGC',
 'HBNC

# fetching data 

we are scraping yahoo finance for the data 

there are some repeat trys to get the right ticker format


.

I implemented some basic multithreading to speed this up 

its not much but it helps

In [5]:
start='01/01/2020' 
end='09/14/2022'

In [6]:
import concurrent.futures 

def get_seiries(ticker,start,end,entry="adjclose"):
    try:
        raw = get_data(ticker.split(".")[0], start_date=start, end_date=end)[entry]
    except: 
        try:
            raw = get_data(ticker.replace(".","-"), start_date=start, end_date=end)[entry]
        except:
            raw = get_data(ticker[0:4], start_date=start, end_date=end)[entry]
    return raw/raw[0]

def get_df(l,start,end,entry="adjclose"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        x=[executor.submit(lambda x: get_seiries(x,start,end,entry=entry),name)for name in l]
        x=[a.result() for a in x]
    for i,c in enumerate(x):
        c.name=l[i]
    return pd.DataFrame(x).transpose()
y=get_df(ticker_list[2000:2100],start,end,entry="adjclose")
y

,CLBR.U,FEBZ,FNVTU,FMAO,INTU,FTHY,KSS,IDHD,IRWD,UGA,...,PSMC,WIP,NTWK,ESACU,SCCE,CGUS,MSTB,TSBK,KBND,NETZ
2020-01-02,NaN,NaN,NaN,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN
2020-01-03,NaN,NaN,NaN,0.976253,0.993312,NaN,1.001629,1.003049,0.993967,1.027974,...,1.001942,1.001087,0.983051,NaN,NaN,NaN,NaN,0.976843,0.997863,NaN
2020-01-06,NaN,NaN,NaN,0.978232,1.000939,NaN,1.019344,1.011733,1.056561,1.024593,...,1.000544,1.005073,0.963680,NaN,NaN,NaN,NaN,0.961839,1.000000,NaN
2020-01-07,NaN,NaN,NaN,0.966689,1.001165,NaN,1.021584,1.000354,1.006033,1.010145,...,1.000388,0.999275,0.932203,NaN,NaN,NaN,NaN,0.958904,1.003206,NaN
2020-01-08,NaN,NaN,NaN,0.975264,1.025248,NaN,1.005498,1.002375,0.991704,0.980941,...,1.000777,0.997645,0.895884,NaN,NaN,NaN,NaN,0.965427,1.005496,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,1.012435,1.076099,0.997024,0.948012,1.654820,0.798128,0.636615,0.867705,0.806184,1.546265,...,0.991097,0.804788,0.888620,1.003980,0.929293,0.911228,1.040355,0.933875,0.972979,1.050112
2022-09-08,1.012435,1.079738,0.997024,0.948357,1.669076,0.800797,0.645646,0.865507,0.827300,1.557947,...,0.987864,0.794823,0.903148,1.003980,0.933333,0.916092,1.046223,0.928978,0.973440,1.062238
2022-09-09,1.008290,1.091694,0.997024,0.958703,1.710857,0.798128,0.660265,0.880891,0.829563,1.626499,...,0.993522,0.803472,0.878935,1.013930,0.943434,0.931496,1.059455,0.944718,0.971132,1.076136
2022-09-12,1.010363,1.097533,0.993056,0.962496,1.731157,0.798128,0.664565,0.885817,0.831071,1.651706,...,0.995948,0.801592,0.907990,1.008955,0.934949,0.940008,1.067062,0.945067,0.988677,1.079376


In [7]:
def time_encode(t):
    first= [(a.day_of_week,a.day/30,a.month/12,(a.year-2020)) for a in t]
    return [[x[i] for x in first] for i in range(4)]
time_encode(y.index[0:5])

[[3, 4, 0, 1, 2],
 [0.06666666666666667, 0.1, 0.2, 0.23333333333333334, 0.26666666666666666],
 [0.08333333333333333,
  0.08333333333333333,
  0.08333333333333333,
  0.08333333333333333,
  0.08333333333333333],
 [0, 0, 0, 0, 0]]

In [8]:
look=layers.StringLookup(vocabulary=ticker_list)
inv_look=layers.StringLookup(vocabulary=ticker_list,invert=True)
tick_embed=layers.Embedding(len(ticker_list)+1,512,embeddings_constraint= tf.keras.constraints.unit_norm)

day_embed=layers.Embedding(7,10)

# secondry testing for errors

In [ ]:
segmented_dfs=[]
shitlist=[]
for i in range(0,len(ticker_list)-100,100):
    try:
        segmented_dfs.append(get_df(ticker_list[i:i+100],start,end,entry="adjclose"))
        print(i)
    except: 
        print(f"error at {i}")
        shitlist.append(i)
#total_df.to_csv(f"prices: {start} to {end}.csv")

In [ ]:
print("yay")

In [ ]:
#get_df(ticker_list[10600],start,end,entry="adjclose")
extra=set(range(10600,len(ticker_list)))
extra

In [ ]:
x=[set(range(i,i+100)) for i in shitlist]
x.append(extra)
x=set.union(*x)
x

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor: 
    t=[executor.submit(lambda x: get_seiries(x,start,end,entry="adjclose"),ticker_list[i]) for i in x]

In [ ]:
del(t)